# Add drive and library

This adds a shared drive where the json file with training data is located, and the library used for preparing data.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)
import sys
drive_path = '/content/gdrive/My Drive/603 Platform for Big Data Processing-Group Project/'
sys.path.append(drive_path)
sys.path.insert(0, f'{drive_path}a-PyTorch-Tutorial-to-Image-Captioning')

Mounted at /content/gdrive/


# Misc settings

Imports misc. packages and sets misc. variables.

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import warnings
warnings.filterwarnings("ignore")
import json

data_path = f'{drive_path}data/training_data/'
data_images_path = f'{data_path}images/'

## Split images

70% train
15% test
15% validate

In [3]:
data_df = pd.read_json(f'{data_path}/train.json')
data_df.head()

,filename,hasBeenUsedForTraining,imgid,isValidCaption,sentences,server
0,51161553607_28d5d4af49_c.jpg,False,0,0,"[{'imgid': 0, 'raw': 'Your photo booth pic at ...",65535
1,51161569167_974e73780f_c.jpg,False,1,0,"[{'imgid': 1, 'raw': 'From RSPB: Key informat...",65535
2,51161577027_efa1a4d7b3_c.jpg,False,2,0,"[{'imgid': 2, 'raw': 'Cute little BOM tattoo &...",65535
3,51161578802_f90d9aa758_c.jpg,False,3,0,"[{'imgid': 3, 'raw': '<a href=""http://maps.sec...",65535
4,51161582827_dbf55d0284_c.jpg,False,4,0,"[{'imgid': 4, 'raw': 'Edited in Prisma app wit...",65535


In [10]:
from sklearn.model_selection import train_test_split

file_names = data_df['filename'].unique()

X_train, X_test = train_test_split(file_names, test_size=0.3, random_state=42)

X_test, X_val = train_test_split(X_test, test_size=0.5, random_state=42)

print(X_train.shape[0])
print(X_test.shape[0])
print(X_val.shape[0])


181
39
39


# Add split property

This adds the split to which each pictures belongs

In [12]:
import string

with open(f'{data_path}/train.json') as f:
  data_dict = json.load(f)

for image_object in data_dict:
  image_name = image_object['filename']
  if image_name in X_val:
    image_object['split'] = 'val'
  elif image_name in X_test:
    image_object['split'] = 'test'
  else:
    image_object['split'] = 'train'

This creates and saves the JSON file in the correct format to feed to `create_input_files` function below.

In [ ]:
dataset = {
    'images': data_dict,
    'dataset': 'flickr_db'
}

del data_dict

train__iteration = 2

with open(f'{drive_path}data/karpathy_json_{train__iteration}.json', 'w') as fp:
    json.dump(dataset, fp)

del dataset

# Create input file

Call a function from a forked respository that creates the file that will be used to train the model.

https://github.com/jorgeso/a-PyTorch-Tutorial-to-Image-Captioning/blob/master/utils.py

In [ ]:
!pip install scipy==1.2

In [ ]:
import utils

utils.create_input_files(dataset='flickr_db',
                       karpathy_json_path=f'{drive_path}data/karpathy_json_{train__iteration}.json',
                       image_folder=data_images_path,
                       captions_per_image=5,
                       min_word_freq=5,
                       output_folder=f'{drive_path}model_output/',
                       max_len=50)

  0%|          | 8/5663 [00:00<01:18, 72.25it/s]


Reading TRAIN images and captions, storing to file...



  1%|          | 9/1214 [00:00<00:14, 81.00it/s]


Reading VAL images and captions, storing to file...



  0%|          | 0/1214 [00:00<?, ?it/s]


Reading TEST images and captions, storing to file...



100%|██████████| 1214/1214 [03:26<00:00,  5.87it/s]
